<a href="https://colab.research.google.com/github/xennygrimmato/word_seq2word_seq/blob/master/src/ccd_word_based_seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install tensorflow

In [0]:
from __future__ import print_function

import os
import json
import pprint
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Input, LSTM, Dense
import numpy as np
import tensorflow as tf

In [4]:
if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)

  with tf.Session(tpu_address) as session:
    devices = session.list_devices()
    
  print('TPU devices:')
  pprint.pprint(devices)

ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!


In [0]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'fra.txt'

In [8]:
# Vectorize the data.
input_texts = []
target_texts = []
input_words = set()
target_words = set()

# File format: <eng-word>\t<french-word>\n
# Get all lines from file
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

input_sequences = []
output_sequences = []
# Go through number of samples you want to train on
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t ' + target_text + ' \n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    input_token_list = input_text.split()
    output_token_list = ['\t'] + target_text.split() + ['\n']
    input_sequences.append(input_token_list)
    output_sequences.append(output_token_list)
    for word in input_token_list:
        if word not in input_words:
            input_words.add(word)
    for word in output_token_list:
        if word not in target_words:
            target_words.add(word)

print(input_sequences)

input_words = sorted(list(input_words))
target_words = sorted(list(target_words))
num_encoder_tokens = len(input_words)
num_decoder_tokens = len(target_words)
max_encoder_seq_length = max([len(seq) for seq in input_sequences])
max_decoder_seq_length = max([len(seq) for seq in output_sequences])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)


input_token_index = dict(
    [(word, i) for i, word in enumerate(input_words)])
target_token_index = dict(
    [(word, i) for i, word in enumerate(target_words)])

print(input_sequences)
print(output_sequences)


[['Go.'], ['Hi.'], ['Run!'], ['Run!'], ['Who?'], ['Wow!'], ['Fire!'], ['Help!'], ['Jump.'], ['Stop!'], ['Stop!'], ['Stop!'], ['Wait!'], ['Wait!'], ['Go', 'on.'], ['Go', 'on.'], ['Go', 'on.'], ['Hello!'], ['Hello!'], ['I', 'see.'], ['I', 'try.'], ['I', 'won!'], ['I', 'won!'], ['I', 'won.'], ['Oh', 'no!'], ['Attack!'], ['Attack!'], ['Cheers!'], ['Cheers!'], ['Cheers!'], ['Cheers!'], ['Get', 'up.'], ['Go', 'now.'], ['Go', 'now.'], ['Go', 'now.'], ['Got', 'it!'], ['Got', 'it!'], ['Got', 'it?'], ['Got', 'it?'], ['Got', 'it?'], ['Hop', 'in.'], ['Hop', 'in.'], ['Hug', 'me.'], ['Hug', 'me.'], ['I', 'fell.'], ['I', 'fell.'], ['I', 'know.'], ['I', 'left.'], ['I', 'left.'], ['I', 'lost.'], ['I', 'paid.'], ["I'm", '19.'], ["I'm", 'OK.'], ["I'm", 'OK.'], ['Listen.'], ['No', 'way!'], ['No', 'way!'], ['No', 'way!'], ['No', 'way!'], ['No', 'way!'], ['No', 'way!'], ['No', 'way!'], ['No', 'way!'], ['No', 'way!'], ['Really?'], ['Really?'], ['Really?'], ['Thanks.'], ['We', 'try.'], ['We', 'won.'], ['We', 

In [10]:
# Prepare data to be ingested by the model
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_sequence, output_sequence) in enumerate(zip(input_sequences, output_sequences)):
    for t, word in enumerate(input_sequence):
        encoder_input_data[i, t, input_token_index[word]] = 1.
    for t, word in enumerate(output_sequence):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[word]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[word]] = 1.
print(input_sequences)
print(output_sequences)

[['Go.'], ['Hi.'], ['Run!'], ['Run!'], ['Who?'], ['Wow!'], ['Fire!'], ['Help!'], ['Jump.'], ['Stop!'], ['Stop!'], ['Stop!'], ['Wait!'], ['Wait!'], ['Go', 'on.'], ['Go', 'on.'], ['Go', 'on.'], ['Hello!'], ['Hello!'], ['I', 'see.'], ['I', 'try.'], ['I', 'won!'], ['I', 'won!'], ['I', 'won.'], ['Oh', 'no!'], ['Attack!'], ['Attack!'], ['Cheers!'], ['Cheers!'], ['Cheers!'], ['Cheers!'], ['Get', 'up.'], ['Go', 'now.'], ['Go', 'now.'], ['Go', 'now.'], ['Got', 'it!'], ['Got', 'it!'], ['Got', 'it?'], ['Got', 'it?'], ['Got', 'it?'], ['Hop', 'in.'], ['Hop', 'in.'], ['Hug', 'me.'], ['Hug', 'me.'], ['I', 'fell.'], ['I', 'fell.'], ['I', 'know.'], ['I', 'left.'], ['I', 'left.'], ['I', 'lost.'], ['I', 'paid.'], ["I'm", '19.'], ["I'm", 'OK.'], ["I'm", 'OK.'], ['Listen.'], ['No', 'way!'], ['No', 'way!'], ['No', 'way!'], ['No', 'way!'], ['No', 'way!'], ['No', 'way!'], ['No', 'way!'], ['No', 'way!'], ['No', 'way!'], ['Really?'], ['Really?'], ['Really?'], ['Thanks.'], ['We', 'try.'], ['We', 'won.'], ['We', 

In [11]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
print(input_sequences)


Instructions for updating:
Colocations handled automatically by placer.
[['Go.'], ['Hi.'], ['Run!'], ['Run!'], ['Who?'], ['Wow!'], ['Fire!'], ['Help!'], ['Jump.'], ['Stop!'], ['Stop!'], ['Stop!'], ['Wait!'], ['Wait!'], ['Go', 'on.'], ['Go', 'on.'], ['Go', 'on.'], ['Hello!'], ['Hello!'], ['I', 'see.'], ['I', 'try.'], ['I', 'won!'], ['I', 'won!'], ['I', 'won.'], ['Oh', 'no!'], ['Attack!'], ['Attack!'], ['Cheers!'], ['Cheers!'], ['Cheers!'], ['Cheers!'], ['Get', 'up.'], ['Go', 'now.'], ['Go', 'now.'], ['Go', 'now.'], ['Got', 'it!'], ['Got', 'it!'], ['Got', 'it?'], ['Got', 'it?'], ['Got', 'it?'], ['Hop', 'in.'], ['Hop', 'in.'], ['Hug', 'me.'], ['Hug', 'me.'], ['I', 'fell.'], ['I', 'fell.'], ['I', 'know.'], ['I', 'left.'], ['I', 'left.'], ['I', 'lost.'], ['I', 'paid.'], ["I'm", '19.'], ["I'm", 'OK.'], ["I'm", 'OK.'], ['Listen.'], ['No', 'way!'], ['No', 'way!'], ['No', 'way!'], ['No', 'way!'], ['No', 'way!'], ['No', 'way!'], ['No', 'way!'], ['No', 'way!'], ['No', 'way!'], ['Really?'], ['Real

In [13]:
# Define optimizer, loss
model.compile(optimizer=tf.train.AdamOptimizer(),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

# Define chcekpoint path
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_weights_only=True,
                                                 verbose=1)

model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=10,
          validation_split=0.2,
          callbacks = [cp_callback])

# Save model
model.save('trained_word_seq_2_word_seq_20percent_validation_split_20_epochs.h5')


Train on 8000 samples, validate on 2000 samples
Epoch 1/10
7936/8000 [============================>.] - ETA: 0s - loss: 2.1636 - acc: 0.0961
Epoch 00001: saving model to training_1/cp.ckpt
Instructions for updating:
Use tf.train.CheckpointManager to manage checkpoints rather than manually editing the Checkpoint proto.
8000/8000 [==============================] - 18s 2ms/sample - loss: 2.1610 - acc: 0.0963 - val_loss: 2.1273 - val_acc: 0.0972
Epoch 2/10
7936/8000 [============================>.] - ETA: 0s - loss: 1.7415 - acc: 0.1078
Epoch 00002: saving model to training_1/cp.ckpt
8000/8000 [==============================] - 14s 2ms/sample - loss: 1.7425 - acc: 0.1078 - val_loss: 2.0650 - val_acc: 0.0986
Epoch 3/10
7936/8000 [============================>.] - ETA: 0s - loss: 1.6247 - acc: 0.1148
Epoch 00003: saving model to training_1/cp.ckpt
8000/8000 [==============================] - 14s 2ms/sample - loss: 1.6242 - acc: 0.1148 - val_loss: 2.0011 - val_acc: 0.1033
Epoch 4/10
7936/8000

In [0]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, word) for word, i in target_token_index.items())


In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    #target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += (sampled_char + ' ')

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


In [16]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_sequences[seq_index])
    print('Decoded sentence:', decoded_sentence)


-
Input sentence: ['Go.']
Decoded sentence: Laissez-moi ! 
-
Input sentence: ['Hi.']
Decoded sentence: Regardez ! 
 
-
Input sentence: ['Run!']
Decoded sentence: Merci ! 
 
-
Input sentence: ['Run!']
Decoded sentence: Merci ! 
 
-
Input sentence: ['Who?']
Decoded sentence: Vous ? 
 
-
Input sentence: ['Wow!']
Decoded sentence: Ça tout. 
 
-
Input sentence: ['Fire!']
Decoded sentence: Sois ceci. 
 
-
Input sentence: ['Help!']
Decoded sentence: Reste tranquille 
-
Input sentence: ['Jump.']
Decoded sentence: Continue 
 
-
Input sentence: ['Stop!']
Decoded sentence: Merci ! 
 
-
Input sentence: ['Stop!']
Decoded sentence: Merci ! 
 
-
Input sentence: ['Stop!']
Decoded sentence: Merci ! 
 
-
Input sentence: ['Wait!']
Decoded sentence: Venez ! 
 
-
Input sentence: ['Wait!']
Decoded sentence: Venez ! 
 
-
Input sentence: ['Go', 'on.']
Decoded sentence: Laissez-moi 
 
-
Input sentence: ['Go', 'on.']
Decoded sentence: Laissez-moi 
 
-
Input sentence: ['Go', 'on.']
Decoded sentence: Laissez-moi 